In [1]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import utils.feature_extractors as utils
import optuna
import joblib

from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from utils.utils import evaluate_classification
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier

d:\Academic\Semester 8\Bio Informatics\druggable-protien-detection\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_map = {
    "train_postive_location":"dataset/TR_pos_SPIDER.txt",
    "train_negative_location":"dataset/TR_neg_SPIDER.txt",
    "test_positive_location":"dataset/TS_pos_SPIDER.txt",
    "test_negative_location":"dataset/TS_neg_SPIDER.txt",
    "model_save_location":"./optimized_models",
    "feat_combo_model_save_location":"./feature_combo_models",
    "final_model_save_location":"./final_model",
    "random_seed":9
}

In [3]:
class ProteinFeatureGenerator:
    SELECTED_FEATURES = ["AAC", "DPC", "RScharge", "RSDHP", "RSpolar"]
    
    def __init__(self, positive_data_file: str, negative_data_file: str, feature_type: str = None) -> None:
        super().__init__()

        # Check feature param
        assert feature_type in ProteinFeatureGenerator.SELECTED_FEATURES or feature_type is None
        self.feature_type = feature_type

        # Data manipulation
        self.positive_data_file = positive_data_file
        self.negative_data_file = negative_data_file

        self.positive_data = utils.read_fasta(self.positive_data_file)
        self.negative_data = utils.read_fasta(self.negative_data_file)
        
        self.data = self.positive_data + self.negative_data
        self.targets = np.array([True]*len(self.positive_data) + [False]*len(self.negative_data))

        self.raw_sequences = [x[1] for x in self.data]
        
        
        # Extract features
        print("Extracting AAC Feature ...")
        self.AAC_feature = utils.AAC(self.data)[0]

        print("Extracting DPC Feature ...")
        self.DPC_feature = utils.DPC(self.data, 0)[0]

        print("Extracting RScharge Feature ...")
        self.RScharge_feature = utils.reducedCHARGE(self.data)
        
        print("Extracting RSDHP Feature ...")
        self.RSDHP_feature = utils.reducedDHP(self.data)
        
        print("Extracting RSpolar Feature ...")
        self.RSpolar_feature = utils.reducedPOLAR(self.data)

    def get_feat_combo(self,selected:list = None):
        
        features =[self.AAC_feature,self.DPC_feature,self.RScharge_feature,
                   self.RSDHP_feature,self.RSpolar_feature]
        
        if selected:
            select_index = sorted([ProteinFeatureGenerator.SELECTED_FEATURES.index(x) for x in selected])
            features = [features[x] for x in select_index]
            
        return np.concatenate(features,axis=-1)
        
    
    def __len__(self) -> int:
        return len(self.data)

In [4]:
train_data = ProteinFeatureGenerator(positive_data_file=config_map["train_postive_location"],negative_data_file=config_map["train_negative_location"])

Extracting AAC Feature ...
Extracting DPC Feature ...
Extracting RScharge Feature ...
Extracting RSDHP Feature ...
Extracting RSpolar Feature ...


In [5]:
X_data = {
    "AAC-DPC-RScharge-RSDHP-RSpolar":train_data.get_feat_combo(["AAC","DPC","RScharge","RSDHP","RSpolar"]),
    "AAC-RScharge-RSDHP-RSpolar":train_data.get_feat_combo(["AAC","RScharge","RSDHP","RSpolar"]),
    "DPC-RScharge-RSDHP-RSpolar":train_data.get_feat_combo(["DPC","RScharge","RSDHP","RSpolar"]),
    "AAC-DPC-RScharge":train_data.get_feat_combo(["AAC","DPC","RScharge"]),
    "AAC-DPC-RSDHP":train_data.get_feat_combo(["AAC","DPC","RSDHP"]),
    "AAC-RSDHP-RSpolar":train_data.get_feat_combo(["AAC","RSDHP","RSpolar"]),
    "RScharge-RSDHP-RSpolar":train_data.get_feat_combo(["RScharge","RSDHP","RSpolar"]),
    "AAC-RSpolar":train_data.get_feat_combo(["AAC","RSpolar"]),
}


data_pipelines = {
    "AAC-DPC-RScharge-RSDHP-RSpolar":make_pipeline(StandardScaler()),
    "AAC-RScharge-RSDHP-RSpolar":make_pipeline(StandardScaler()),
    "DPC-RScharge-RSDHP-RSpolar":make_pipeline(StandardScaler()),
    "AAC-DPC-RScharge":make_pipeline(StandardScaler()),
    "AAC-DPC-RSDHP":make_pipeline(StandardScaler()),
    "AAC-RSDHP-RSpolar":make_pipeline(StandardScaler()),
    "RScharge-RSDHP-RSpolar":make_pipeline(StandardScaler()),
    "AAC-RSpolar":make_pipeline(StandardScaler()),
}

In [6]:
best_feature_combo = ""
best_acc = 0
for feature_type in X_data.keys():
    print(f"Feature Combo :- {feature_type} | Model :- SVC")
    
    os.makedirs(os.path.join(config_map["feat_combo_model_save_location"],feature_type,"SVC"),exist_ok=True)
    model_dir = os.path.join(config_map["feat_combo_model_save_location"],feature_type,"SVC")
    
    X,y = X_data[feature_type],train_data.targets
    X,y = shuffle(X,y,random_state=config_map["random_seed"])
    X = data_pipelines[feature_type].fit_transform(X,y)
    
    clf = SVC()
    y_pred = cross_val_predict(clf, X, y, cv=5)
    
    result_values = evaluate_classification(y_pred,y,class_names=["Not Druggable","Druggable"],save_outputs=model_dir)
    
    clf.fit(X,y)
    
    print(result_values)
    # Select best combo to optimize
    if float(result_values["accuracy"])>best_acc:
        best_acc = result_values["accuracy"]
        best_feature_combo = feature_type
    joblib.dump(data_pipelines[feature_type], os.path.join(model_dir,"pipeline.sav"))
    joblib.dump(clf, os.path.join(model_dir,"model.sav"))
    print("\n")
    
print("Best Feature Combo:",best_feature_combo)
print("Best Accuracy:",str(best_acc))

Feature Combo :- AAC-DPC-RScharge-RSDHP-RSpolar | Model :- SVC
{'accuracy': 0.9040125885129819, 'sensitivity': 0.8879803761242846, 'specificity': 0.9188779378316907, 'precision': 0.9043767169124066, 'f1': 0.9037761748264941}


Feature Combo :- AAC-RScharge-RSDHP-RSpolar | Model :- SVC
{'accuracy': 0.8941778127458694, 'sensitivity': 0.8765331152902698, 'specificity': 0.910538286580743, 'precision': 0.8945769728009547, 'f1': 0.89390458490862}


Feature Combo :- DPC-RScharge-RSDHP-RSpolar | Model :- SVC
{'accuracy': 0.9016522423288749, 'sensitivity': 0.8855273916598528, 'specificity': 0.9166034874905231, 'precision': 0.902007447694949, 'f1': 0.90141001519108}


Feature Combo :- AAC-DPC-RScharge | Model :- SVC
{'accuracy': 0.8977183320220299, 'sensitivity': 0.8789860997547015, 'specificity': 0.9150871872630781, 'precision': 0.898209694818094, 'f1': 0.8974417861254993}


Feature Combo :- AAC-DPC-RSDHP | Model :- SVC
{'accuracy': 0.9008654602675059, 'sensitivity': 0.8863450531479967, 'specif

## Optimizing Best Model

In [7]:
# Get the clf and best feature combo
model = "SVC"
best_feature_combo = best_feature_combo
    
# Print the current model and feature type being trained
print(f"Feature Type: {best_feature_combo} | Training Model: {model}")
    
# Shuffle the data and apply the data pipeline
X, y = shuffle(X_data[best_feature_combo], train_data.targets, random_state=config_map["random_seed"])
data_pipeline = data_pipelines[best_feature_combo]
X = data_pipeline.fit_transform(X, y)
    
# Define the objective function for the Optuna optimization
def obj_func_svc(trial: optuna.trial) -> SVC:
    c = trial.suggest_float('C', 1e-10, 1e10, log=True)
    kernel = 'rbf'
    gamma = 'auto'
        
    classifier = SVC(
        C=c, 
        kernel=kernel,
        class_weight={1: 0.482, 0: 0.518},
        gamma=gamma,
    )
        
    return classifier
def objective(trial):
    clf = obj_func_svc(trial)
    y_pred = cross_val_predict(clf, X, y, cv=5)
    result_values = evaluate_classification(
        y_pred, y, class_names=["Not Druggable", "Druggable"],
        save_outputs=None
    )
    return result_values["f1"]


# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and f1 score obtained
print('Best F1 Score: {}'.format(study.best_trial.value))
print("Best Hyperparameters: {}".format(study.best_trial.params))


# Save the best hyperparameters, study object, data pipeline object, and trained model
params = study.best_trial.params
model_dir = os.path.join(config_map["final_model_save_location"], best_feature_combo, model)
os.makedirs(model_dir, exist_ok=True)

with open(os.path.join(model_dir, "hyperparams.json"), "w") as f:
    json.dump(params, f)

classifier = SVC(**params)
classifier.fit(X, y)

joblib.dump(classifier, os.path.join(model_dir, "model.sav"))
joblib.dump(study, os.path.join(model_dir, "optuna_study.sav"))
joblib.dump(data_pipeline, os.path.join(model_dir, "pipeline.sav"))

# Print newlines to separate the outputs for each feature type
print("\n")


[I 2023-05-12 12:04:53,390] A new study created in memory with name: no-name-6884c039-4671-4e93-87aa-0872d69f60e4


Feature Type: AAC-DPC-RScharge-RSDHP-RSpolar | Training Model: SVC


[I 2023-05-12 12:04:58,542] Trial 0 finished with value: 0.8935627764678735 and parameters: {'C': 1679.2398987701054}. Best is trial 0 with value: 0.8935627764678735.
[I 2023-05-12 12:05:02,956] Trial 1 finished with value: 0.8935627764678735 and parameters: {'C': 145622367.14498752}. Best is trial 0 with value: 0.8935627764678735.
[I 2023-05-12 12:05:07,311] Trial 2 finished with value: 0.8935627764678735 and parameters: {'C': 2766914888.1091757}. Best is trial 0 with value: 0.8935627764678735.
[I 2023-05-12 12:05:12,184] Trial 3 finished with value: 0.8849780730909407 and parameters: {'C': 0.42555561238599443}. Best is trial 0 with value: 0.8935627764678735.
[I 2023-05-12 12:05:16,503] Trial 4 finished with value: 0.8939607870868114 and parameters: {'C': 91.34022639390524}. Best is trial 4 with value: 0.8939607870868114.
[I 2023-05-12 12:05:20,856] Trial 5 finished with value: 0.9077197742188509 and parameters: {'C': 4.527116132785931}. Best is trial 5 with value: 0.9077197742188509.

Best F1 Score: 0.9085084940973223
Best Hyperparameters: {'C': 3.5441483407548477}




## Test

In [8]:
test_data = ProteinFeatureGenerator(positive_data_file=config_map["test_positive_location"],negative_data_file=config_map["test_negative_location"])

Extracting AAC Feature ...
Extracting DPC Feature ...
Extracting RScharge Feature ...
Extracting RSDHP Feature ...
Extracting RSpolar Feature ...


In [9]:
X_test = {
    "AAC-DPC-RScharge-RSDHP-RSpolar":test_data.get_feat_combo(["AAC","DPC","RScharge","RSDHP","RSpolar"]),
    "AAC-RScharge-RSDHP-RSpolar":test_data.get_feat_combo(["AAC","RScharge","RSDHP","RSpolar"]),
    "DPC-RScharge-RSDHP-RSpolar":test_data.get_feat_combo(["DPC","RScharge","RSDHP","RSpolar"]),
    "AAC-DPC-RScharge":test_data.get_feat_combo(["AAC","DPC","RScharge"]),
    "AAC-DPC-RSDHP":test_data.get_feat_combo(["AAC","DPC","RSDHP"]),
    "AAC-RSDHP-RSpolar":test_data.get_feat_combo(["AAC","RSDHP","RSpolar"]),
    "RScharge-RSDHP-RSpolar":test_data.get_feat_combo(["RScharge","RSDHP","RSpolar"]),
    "AAC-RSpolar":test_data.get_feat_combo(["AAC","RSpolar"]),
}

In [11]:
feature_type = best_feature_combo
print(f" Feature Type :- {feature_type} | Model :- SVC")
    
model_dir = os.path.join(config_map["final_model_save_location"],feature_type,"SVC")
os.makedirs(model_dir,exist_ok=True)


pipeline = joblib.load(os.path.join(config_map["final_model_save_location"],feature_type,"SVC","pipeline.sav"))
clf = joblib.load(os.path.join(config_map["final_model_save_location"],feature_type,"SVC","model.sav"))

X,y = X_test[feature_type],test_data.targets

X,y = shuffle(X,y,random_state=config_map["random_seed"])
X = pipeline.transform(X)

y_pred = clf.predict(X)

result_values = evaluate_classification(y_pred,y,class_names=["Not Druggable","Druggable"],save_outputs=model_dir)

print(result_values)
print("\n\n")

 Feature Type :- AAC-DPC-RScharge-RSDHP-RSpolar | Model :- SVC
{'accuracy': 0.8698481561822126, 'sensitivity': 0.7857142857142857, 'specificity': 0.9493670886075949, 'precision': 0.8801730184708908, 'f1': 0.8683609366076528}



